In [26]:
from collections import defaultdict
from functools import wraps
from pathlib import Path
import glob
from netCDF4 import Dataset
import re

def cfrad(input_dir, output_dir, dualpol=False, gridder=False, plot=None):
    '''
        input_dir(str): Enter path of single sweep data directory,
        output_dir(str): Enter the path for output data,
        dualpol(bool): True, False. (If the data contains dual-pol products e.g., ZDR, RHOHV),
        gridder(bool): True, False,
        plot(str): 'REF', 'VELH', 'WIDTH', 'ALL',
    '''
    def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
        return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

    def create_output_dir(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            output_dir = kwargs['output_dir']
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            return func(*args, **kwargs)
        return wrapper

    @create_output_dir
    def process_files(input_dir, output_dir, dualpol, **kwargs):
        files = [entry.name for entry in os.scandir(input_dir) if 'nc' in entry.name]
        # Sort the list of file names using the natural_sort_key function
        files_list = sorted(files, key=natural_sort_key)
        print('Number of files: ', len(files_list))
        files_grouped = [files_list[i:i+10] for i in range(0, len(files_list), 10)]
        print('Total number of files will be created: ', len(files_grouped))
        print('Merging all scans in one file')

        data = defaultdict(list)
        for i, files in enumerate(files_grouped):
            for j, file in enumerate(files):
                with Dataset(file) as ds:
                    data['radialAzim'].extend(ds.variables['radialAzim'][:])
                    data['radialTime'].extend(ds.variables['radialTime'][:])
                    data['radialElev'].extend(ds.variables['radialElev'][:])
                    data['Z'].extend(ds.variables['Z'][:])
                    data['V'].extend(ds.variables['V'][:])
                    data['W'].extend(ds.variables['W'][:])
                    data['elevationNumber'].append(ds.variables['elevationNumber'][:])
                    data['nyquist'].append(ds.variables['nyquist'][:])
                    data['unambigRange'].append(ds.variables['unambigRange'][:])
                    if dualpol:
                        data['ZDR'].extend(ds.variables['ZDR'][:])
                        data['PHIDP'].extend(ds.variables['PHIDP'][:])
                        data['KDP'].extend(ds.variables['KDP'][:])
                        data['SQI'].extend(ds.variables['SQI'][:])
                        data['RHOHV'].extend(ds.variables['RHOHV'][:])
                        data['HCLASS'].extend(ds.variables['HCLASS'][:])

                fname = files[0].split("\\")[-1] if "\\" in files[0] else files[0].split("/")[-1]
                print(f'Creating file {i+1}/{len(files_grouped)}: {fname}')
                with Dataset(f'{output_dir}/{fname}', 'w') as out_ds:
                    out_ds.createDimension('time', None)
                    out_ds.createDimension('range', len(data['Z']))
                    out_ds.createDimension('radial', len(data['radialAzim']))
                    out_ds.createVariable('time', 'f8', ('time',))
                    out_ds.createVariable('range', 'f8', ('range',))
                    out_ds.createVariable('radial', 'f8', ('radial',))
                    out_ds.createVariable('Z', 'f8', ('time', 'range'))
                    out_ds.createVariable('V', 'f8', ('time', 'range'))
                    out_ds.createVariable('W', 'f8', ('time', 'range'))
                    out_ds.createVariable('radialAzim', 'f8', ('radial',))
                    out_ds.createVariable('radialTime', 'f8', ('radial',))
                    out_ds.createVariable('radialElev', 'f8', ('radial',))
                    out_ds.createVariable('elevationNumber', 'i4', ('radial',))
                    out_ds.createVariable('nyquist', 'f8', ('radial',))
                    out_ds.createVariable('unambigRange', 'f8', ('radial',))
                    if dualpol:
                        out_ds.createVariable('ZDR', 'f8', ('time', 'range'))
                        out_ds.createVariable('PHIDP', 'f8', ('time', 'range'))
                        out_ds.createVariable('KDP', 'f8', ('time', 'range'))
                        out_ds.createVariable('SQI', 'f8', ('time', 'range'))
                        out_ds.createVariable('RHOHV', 'f8', ('time', 'range'))
                        out_ds.createVariable('HCLASS', 'f8', ('time', 'range'))

                    out_ds.variables['time'][:] = data['radialTime']
                    out_ds.variables['range'][:] = data['unambigRange'][0]
                    out_ds.variables['radial'][:] = data['elevationNumber']
                    out_ds.variables['Z'][:] = data['Z']
                    out_ds.variables['V'][:] = data['V']
                    out_ds.variables['W'][:] = data['W']
                    out_ds.variables['radialAzim'][:] = data['radialAzim']
                    out_ds.variables['radialTime'][:] = data['radialTime']
                    out_ds.variables['radialElev'][:] = data['radialElev']
                    out_ds.variables['elevationNumber'][:] = data['elevationNumber']
                    out_ds.variables['nyquist'][:] = data['nyquist']
                    out_ds.variables['unambigRange'][:] = data['unambigRange']
                    if dualpol:
                        out_ds.variables['ZDR'][:] = data['ZDR']
                        out_ds.variables['PHIDP'][:] = data['PHIDP']
                        out_ds.variables['KDP'][:] = data['KDP']
                        out_ds.variables['SQI'][:] = data['SQI']
                        out_ds.variables['RHOHV'][:] = data['RHOHV']
                        out_ds.variables['HCLASS'][:] = data['HCLASS']

                if gridder:
                    print(f'Gridding file {i+1}/{len(files_grouped)}: {fname}')
                    # Load the radar data
                    with Dataset(f'{output_dir}/{fname}', 'r') as radar_ds:
                        radar = pyart.io.netcdf.read_cfradial(radar_ds)
                    # Create the grid
                    grid = get_grid(radar)
                    # Save the grid to a file
                    pyart.io.write_grid(f'{output_dir}/{fname}_grid.nc', grid)
                    

                if plot:
                    print(f'Plotting file {i+1}/{len(files_grouped)}: {fname}')
                    # Load the radar data
                    with Dataset(f'{output_dir}/{fname}', 'r') as radar_ds:
                        radar = pyart.io.netcdf.read_cfradial(radar_ds)
                    # Plot the CAPPI
                    plot_cappi(radar, 'Z', height=10)
        process_files(input_dir, output_dir, dualpol)

In [27]:
def cfrad(input_dir, output_dir, dualpol=False, gridder=False, plot=None):
    # ... function definition and other code here ...

    @create_output_dir
    def process_files(input_dir, output_dir, dualpol, **kwargs):
        # ... function definition and other code here ...

            if gridder:
                print(f'Gridding file {i+1}/{len(files_grouped)}: {fname}')
                # Load the radar data
                with Dataset(f'{output_dir}/{fname}', 'r') as radar_ds:
                    radar = pyart.io.netcdf.read_cfradial(radar_ds)
                # Create the grid
                grid = get_grid(radar)
                # Save the grid to a file
                pyart.io.write_grid(f'{output_dir}/{fname}_grid.nc', grid)

            if plot:
                print(f'Plotting file {i+1}/{len(files_grouped)}: {fname}')
                # ... code to plot the data here ...

    process_files(input_dir, output_dir, dualpol)

In [28]:
def cfrad(input_dir, output_dir, dualpol=False, gridder=False, plot=None):
    # ... function definition and other code here ...

    @create_output_dir
    def process_files(input_dir, output_dir, dualpol, **kwargs):
        # ... function definition and other code here ...

            if plot:
                print(f'Plotting file {i+1}/{len(files_grouped)}: {fname}')
                # Load the radar data
                with Dataset(f'{output_dir}/{fname}', 'r') as radar_ds:
                    radar = pyart.io.netcdf.read_cfradial(radar_ds)
                # Plot the CAPPI
                plot_cappi(radar, 'Z', height=5)

    process_files(input_dir, output_dir, dualpol)

In [29]:
def plot_cappi(grid, prod, **kwargs):
    """Plots CAPPI.

    Parameters
    ----------
    grid : pyart.core.grid.Grid
        Py-ART grid object.
    prod : str
        Radar product, e.g., "REF", "VELH", "WIDTH", or "ALL".
    **kwargs : optional keyword arguments
        Passed to matplotlib.pyplot.contourf.

    Returns
    -------
    fig : matplotlib.figure.Figure
        Figure object.
    """
    # map prod values to configurations
    config = {
        'REF': {
            'cmap': 'pyart_NWSRef',
            'levels': range(0, 70),
            'stitle': 'Max-Z',
            'munit': 'dBZ'
        },
        'VELH': {
            'cmap': 'pyart_NWSVel',
            'levels': range(-30, 31),
            'stitle': 'Max-V',
            'munit': 'm/s'
        },
        'WIDTH': {
            'cmap': 'pyart_NWS_SPW',
            'levels': np.linspace(0, 5, 9),
            'stitle': 'Max-W',
            'munit': 'm/s'
        }
    }

    # retrieve configuration for prod
    cmap = config[prod]['cmap']
    levels = config[prod]['levels']
    stitle = config[prod]['stitle']
    munit = config[prod]['munit']

    # Extract fields and metadata from the grid
    max_c = grid.fields[prod]['data'].max(axis=0)
    max_x = grid.fields[prod]['data'].max(axis=1)
    max_y = grid.fields[prod]['data'].max(axis=2).T
    trgx = grid.x['data']
    trgy = grid.y['data']
    trgz = grid.z['data']
    origin_longitude = grid.origin_longitude['data'][0]
    origin_latitude = grid.origin_latitude['data'][0]

    # Create the figure
    fig = plt.figure(figsize=(10.3, 10))
    # define axes
    left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
    ax_xy = plt.axes((left, bottom, width, width),
                     projection=ccrs.AzimuthalEquidistant(
                         grid.origin_longitude['data'][0],
                         grid.origin_latitude['data'][0]))

    left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
    left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
    ax_xy = plt.axes((left, bottom, width, width),
                     projection=ccrs.AzimuthalEquidistant(origin_longitude, origin_latitude))
    ax_x = plt.axes((left, bottom + width, width, height))
    ax_y = plt.axes((left + width, bottom, height, width))
    ax_cnr = plt.axes((left+width, bottom + width, left+left, height))
    ax_cb = plt.axes((left + width + height + 0.02, bottom, 0.02, width))
    # set axis label formatters
    ax_x.xaxis.set_major_formatter(NullFormatter())
    ax_y.yaxis.set_major_formatter(NullFormatter())
    ax_cnr.yaxis.set_major_formatter(NullFormatter())
    ax_cnr.xaxis.set_major_formatter(NullFormatter())

    # label axes
    ax_x.set_ylabel("Height AMSL (km)", fontsize=15)
    ax_y.set_xlabel("Height AMSL (km)", fontsize=15)

    # draw CAPPI
    plt.sca(ax_xy)
    xy = ax_xy.contourf(trgx, trgy, max_c, cmap=cmap, levels=levels, **kwargs)
    gl = ax_xy.gridlines(
        crs=ccrs.PlateCarree(),
        linewidth=1, color='black', alpha=0.5,
        linestyle='--', draw_labels=True)
    ax_xy.coastlines(linewidth=1)
    ax_xy.set_title(stitle)
    # draw vertical plane
    plt.sca(ax_x)
    ax_x.plot(max_x, trgz)
    ax_x.grid(True)
    ax_x.set_ylim([0, 15])
    # draw horizontal plane
    plt.sca(ax_y)
    ax_y.plot(trgz, max_y)
    ax_y.grid(True)
    ax_y.set_ylim([0, 15])
    # draw cross section
    plt.sca(ax_cnr)
    ax_cnr.plot(max_c[:, 250], trgz)
    ax_cnr.grid(True)
    ax_cnr.set_ylim([0, 15])
    ax_cnr.set_xlim([10, 60])
    # draw colorbar
    plt.sca(ax_cb)
    cb = plt.colorbar(xy, cax=ax_cb, orientation='vertical')
    cb.set_label(f'{prod} ({munit})')
    # show plot
    plt.show()

In [30]:
import re
import os
from collections import defaultdict
from functools import wraps
from pathlib import Path
import glob
from netCDF4 import Dataset

class CFrad:
    def __init__(self, input_dir, output_dir, dualpol=False, gridder=False, plot=None):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.dualpol = dualpol
        self.gridder = gridder
        self.plot = plot
        self.files_list = None
        self.files_grouped = None
        self.data = defaultdict(list)

    def natural_sort_key(self, s, _nsre=re.compile('([0-9]+)')):
        return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

    def create_output_dir(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            Path(self.output_dir).mkdir(parents=True, exist_ok=True)
            return func(self, *args, **kwargs)
        return wrapper

    @create_output_dir
    def process_files(self):
        self.files_list = sorted(glob.glob(os.path.join(self.input_dir, '*nc*')), key=self.natural_sort_key)
        print('Number of files: ', len(self.files_list))
        self.files_grouped = [self.files_list[i:i+10] for i in range(0, len(self.files_list), 10)]
        print('Total number of files will be created: ', len(self.files_grouped))
        print('Merging all scans in one file')

        for i, files in enumerate(self.files_grouped):
            for j, file in enumerate(files):
                with Dataset(file) as ds:
                    self.data['radialAzim'].extend(ds.variables['radialAzim'][:])
                    self.data['radialTime'].extend(ds.variables['radialTime'][:])
                    self.data['radialElev'].extend(ds.variables['radialElev'][:])
                    self.data['Z'].extend(ds.variables['Z'][:])
                    self.data['V'].extend(ds.variables['V'][:])
                    self.data['W'].extend(ds.variables['W'][:])
                   


In [31]:
def plot_cappi(self, grid, prod, **kwargs):
    # map prod values to configurations
    config = {
        'REF': {
            'cmap': 'pyart_NWSRef',
            'levels': range(10, 60),
            'stitle': 'Max-Z',
            'munit': 'dBZ'
        },
        'VELH': {
            'cmap': 'pyart_NWSVel',
            'levels': range(-30, 31),
            'stitle': 'Max-V',
            'munit': 'm/s'
        },
        'WIDTH': {
            'cmap': 'pyart_NWS_SPW',
            'levels': np.linspace(0, 5, 9),
            'stitle': 'Max-W',
            'munit': 'm/s'
        }
    }

    # retrieve configuration for prod
    cmap = config[prod]['cmap']
    levels = config[prod]['levels']
    stitle = config[prod]['stitle']
    munit = config[prod]['munit']

    max_c = grid.fields[prod]['data'].max(axis=0)
    max_x = grid.fields[prod]['data'].max(axis=1)
    max_y = grid.fields[prod]['data'].max(axis=2).T
    trgx = grid.x['data']
    trgy = grid.y['data']
    trgz = grid.z['data']
    print('Generating Figures of Max-CAPPI, please be patient!')
    fig = plt.figure(figsize=(10.3, 10))
    # define axes
    left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
    left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
    ax_xy = plt.axes((left, bottom, width, width),
                     projection=ccrs.AzimuthalEquidistant(
                         grid.origin_longitude['data'][0],
                         grid.origin_latitude['data'][0]))
    ax_x = plt.axes((left, bottom + width, width, height))
    ax_y = plt.axes((left + width, bottom, height, width))
    ax_cnr = plt.axes((left+width, bottom + width, left+left, height))
    ax_cb = plt.axes((left + width + height + 0.02, bottom, 0.02, width))

    # set axis label formatters
    ax_x.xaxis.set_major_formatter(NullFormatter())
    ax_y.yaxis.set_major_formatter(NullFormatter())
    ax_cnr.yaxis.set_major_formatter(NullFormatter())
    ax_cnr.xaxis.set_major_formatter(NullFormatter())

    # label axes
    # ax_xy.set_xlabel("Longitude [°E]",fontsize=15)
    # ax_xy.set_ylabel("Latitude [°N]",fontsize=15)
    # ax_x.set_xlabel("")
    ax_x.set_ylabel("Height AMSL (km)", fontsize=15)
    # ax_y.set_ylabel("")
    ax_y.set_xlabel("Height AMSL (km)", fontsize=15)

    # draw CAPPI
    plt.sca(ax_xy)
    xy = ax_xy.contourf(trgx, trgy, max_c, cmap=cmap, levels=levels, **kwargs)
    gl = ax_xy.gridlines(
        crs=ccrs.PlateCarree(),
        linewidth=1, color='black', alpha=0.5,
        linestyle='--', draw_labels=True
                        )
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_bottom = True
    dl.xlabels_right = False
    gl.xlines = False
    gl.ylines = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    # draw cross section X
    plt.sca(ax_x)
    ax_x.contourf(max_x, trgz, cmap=cmap, levels=levels)
    ax_x.set_ylim(trgz[0], trgz[-1])
    # draw cross section Y
    plt.sca(ax_y)
    ax_y.contourf(max_y, trgz, cmap=cmap, levels=levels)
    ax_y.set_ylim(trgz[0], trgz[-1])
    # draw colorbar
    plt.sca(ax_cb)
    cb = plt.colorbar(xy, cax=ax_cb, orientation="vertical")
    cb.set_label(f'[{munit}]', fontsize=15)
    cb.ax.tick_params(labelsize=15)
    # draw corner colorbar
    plt.sca(ax_cnr)
    plt.hist2d(max_c.flatten(), max_x.flatten(), bins=100,
               cmap=cmap, norm=LogNorm())
    plt.xlabel(f'Max-X [{munit}]', fontsize=15)
    plt.ylabel(f'Max-Y [{munit}]', fontsize=15)
    plt.title(f'{stitle}', fontsize=15)
    plt.xlabel(f'Max-X [{munit}]', fontsize=15)
    plt.ylabel(f'Max-Y [{munit}]', fontsize=15)
    plt.title(f'{stitle}', fontsize=15)
    plt.colorbar()
    plt.show()

In [33]:
import re
import os
from collections import defaultdict
from functools import wraps
from pathlib import Path
import glob
from netCDF4 import Dataset

class Cfradial:
    def __init__(self, input_dir, output_dir, dualpol=False, gridder=False, plot=None):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.dualpol = dualpol
        self.gridder = gridder
        self.plot = plot
        self.files_list = None
        self.files_grouped = None
        self.data = defaultdict(list)

    def natural_sort_key(self, s, _nsre=re.compile('([0-9]+)')):
        return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

    def create_output_dir(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            Path(self.output_dir).mkdir(parents=True, exist_ok=True)
            return func(self, *args, **kwargs)
        return wrapper

    @create_output_dir
    def process_files(self):
        self.files_list = sorted(glob.glob(os.path.join(self.input_dir, '*nc*')), key=self.natural_sort_key)
        print('Number of files: ', len(self.files_list))
        self.files_grouped = [self.files_list[i:i+10] for i in range(0, len(self.files_list), 10)]
        print('Total number of files will be created: ', len(self.files_grouped))
        print('Merging all scans in one file')

        for i, files in enumerate(self.files_grouped):
            for j, file in enumerate(files):
                with Dataset(file) as ds:
                    self.data['radialAzim'].extend(ds.variables['radialAzim'][:])
                    self.data['radialTime'].extend(ds.variables['radialTime'][:])
                    self.data['radialElev'].extend(ds.variables['radialElev'][:])
                    self.data['Z'].extend(ds.variables['Z'][:])
                    self.data['V'].extend(ds.variables['V'][:])
                    self.data['W'].extend(ds.variables['W'][:])
                    self.data['elevationNumber'].append(ds.variables['elevationNumber'][:])
                    self.data['nyquist'].append(ds.variables['nyquist'][:])
                    self.data['unambigRange'].append(ds.variables['unambigRange'][:])
                    if self.dualpol:
                        self.data['ZDR'].extend(ds.variables['ZDR'][:])
                        self.data['PHIDP'].extend(ds.variables['PHIDP'][:])
                        self.data['KDP'].extend(ds.variables['KDP'][:])
                        self.data['SQI'].extend(ds.variables['SQI'][:])
                        self.data['RHOHV'].extend(ds.variables['RHOHV'][:])
                        self.data['HCLASS'].extend(ds.variables['HCLASS'][:])

                    fname = files[0].split("\\")[-1] if "\\" in files[0] else files[0].split("/")[-1]
                    print(f'Creating file {i+1}/{len(self.files_grouped)}: {fname}')
                    with Dataset(os.path.join(self.output_dir, fname), 'w') as out_ds:
                        out_ds.createDimension('time', None)
                        out_ds.createDimension('range', len(self.data['Z']))
                        out_ds.createDimension('radial', len(self.data['radialAzim']))

                        out_ds.createVariable('radialAzim', 'f8', ('radial'))
                        out_ds.createVariable('radialTime', 'f8', ('radial'))
                        out_ds.createVariable('radialElev', 'f8', ('radial'))
                        out_ds.createVariable('Z', 'f8', ('time', 'range'))
                        out_ds.createVariable('V', 'f8', ('time', 'range'))
                        out_ds.createVariable('W', 'f8', ('time', 'range'))
                        out_ds.createVariable('elevationNumber', 'i4', ('time'))
                        out_ds.createVariable('nyquist', 'f4', ('time'))
                        out_ds.createVariable('unambigRange', 'f4', ('time'))
                        if self.dualpol:
                            out_ds.createVariable('ZDR', 'f8', ('time', 'range'))
                            out_ds.createVariable('PHIDP', 'f8', ('time', 'range'))
                            out_ds.createVariable('KDP', 'f8', ('time', 'range'))
                            out_ds.createVariable('SQI', 'f8', ('time', 'range'))
                            out_ds.createVariable('RHOHV', 'f8', ('time', 'range'))
                            out_ds.createVariable('HCLASS', 'f8', ('time', 'range'))

                        out_ds.variables['radialAzim'][:] = self.data['radialAzim']
                        out_ds.variables['radialTime'][:] = self.data['radialTime']
                        out_ds.variables['radialElev'][:] = self.data['radialElev']
                        out_ds.variables['Z'][:, :] = self.data['Z']
                        out_ds.variables['V'][:, :] = self.data['V']
                        out_ds.variables['W'][:, :] = self.data['W']
                        out_ds.variables['elevationNumber'][:] = self.data['elevationNumber']
                        out_ds.variables['nyquist'][:] = self.data['nyquist']
                        out_ds.variables['unambigRange'][:] = self.data['unambigRange']
                        if self.dualpol:
                            out_ds.variables['ZDR'][:, :] = self.data['ZDR']
                            out_ds.variables['PHIDP'][:, :] = self.data['PHIDP']
                            out_ds.variables['KDP'][:, :] = self.data['KDP']
                            out_ds.variables['SQI'][:, :] = self.data['SQI']
                            out_ds.variables['RHOHV'][:, :] = self.data['RHOHV']
                            out_ds.variables['HCLASS'][:, :] = self.data['HCLASS']

                self.data = defaultdict(list)

        def grid_data(self):
            if self.gridder:
                print('Gridding data')
                import pyart
                def get_grid(radar, grid_shape=(30, 500, 500), height=15, length=250):
                    """
                    Returns grid object from radar object.
                    grid_shape=(60, 500, 500), no. of bins of z,y,x respectively.
                    height:(int) = 15, height in km
                    length:(int) = 250, horizontal length in km
                    """
                    grid = pyart.map.grid_from_radars(
                        radar, grid_shape=grid_shape,
                        grid_limits=((radar.altitude['data'][0], height * 1e3),
                                     (-length * 1e3, length * 1e3),
                                     (-length * 1e3, length*1e3)),
                        fields=radar.fields.keys(),
                        weighting_function='Barnes2',
                        min_radius=length)
                    return grid

                from pyart.io.cfradial import read_cfradial
                from pyart.core.radar import Radar
                import numpy as np
                fname = os.path.join(self.output_dir, self.files_list[0].split("\\")[-1] if "\\" in self.files_list[0] else self.files_list[0].split("/")[-1])
                radar = Radar.from_ncdata(read_cfradial(fname))
                grid = get_grid(radar)
        def plot_cappi(self, grid, prod, **kwargs):
        # map prod values to configurations
            config = {
                'REF': {
                    'cmap': 'pyart_NWSRef',
                    'levels': range(10, 60),
                    'stitle': 'Max-Z',
                    'munit': 'dBZ'
                },
                'VELH': {
                    'cmap': 'pyart_NWSVel',
                    'levels': range(-30, 31),
                    'stitle': 'Max-V',
                    'munit': 'm/s'
                },
                'WIDTH': {
                    'cmap': 'pyart_NWS_SPW',
                    'levels': np.linspace(0, 5, 9),
                    'stitle': 'Max-W',
                    'munit': 'm/s'
                }
            }

            # retrieve configuration for prod
            cmap = config[prod]['cmap']
            levels = config[prod]['levels']
            stitle = config[prod]['stitle']
            munit = config[prod]['munit']

            max_c = grid.fields[prod]['data'].max(axis=0)
            max_x = grid.fields[prod]['data'].max(axis=1)
            max_y = grid.fields[prod]['data'].max(axis=2).T
            trgx = grid.x['data']
            trgy = grid.y['data']
            trgz = grid.z['data']
            print('Generating Figures of Max-CAPPI, please be patient!')
            fig = plt.figure(figsize=(10.3, 10))
            # define axes
            left, bottom, width, height = 0.1, 0.1, 0.6, 0.2
            ax_xy = plt.axes((left, bottom, width, width),
                             projection=ccrs.AzimuthalEquidistant(
                                 grid.origin_longitude['data'][0],
                                 grid.origin_latitude['data'][0]))
            ax_x = plt.axes((left, bottom + width, width, height))
            ax_y = plt.axes((left + width, bottom, height, width))
            ax_cnr = plt.axes((left+width, bottom + width, left+left, height))
            ax_cb = plt.axes((left + width + height + 0.02, bottom, 0.02, width))

            # set axis label formatters
            ax_x.xaxis.set_major_formatter(NullFormatter())
            ax_y.yaxis.set_major_formatter(NullFormatter())
            ax_cnr.yaxis.set_major_formatter(NullFormatter())
            ax_cnr.xaxis.set_major_formatter(NullFormatter())

            # label axes
            # ax_xy.set_xlabel("Longitude [°E]",fontsize=15)
            # ax_xy.set_ylabel("Latitude [°N]",fontsize=15)
            # ax_x.set_xlabel("")
            ax_x.set_ylabel("Height AMSL (km)", fontsize=15)
            # ax_y.set_ylabel("")
            ax_y.set_xlabel("Height AMSL (km)", fontsize=15)

            # draw CAPPI
            plt.sca(ax_xy)
            xy = ax_xy.contourf(trgx, trgy, max_c, cmap=cmap, levels=levels, **kwargs)
            gl = ax_xy.gridlines(
                crs=ccrs.PlateCarree(),
                linewidth=1, color='black', alpha=0.5,
                linestyle='--', draw_labels=True
                                )
            gl.xlabels_top = False
            gl.ylabels_left = True
            gl.ylabels_bottom = True
            dl.xlabels_right = False
            gl.xlines = False
            gl.ylines = False
            gl.xformatter = LONGITUDE_FORMATTER
            gl.yformatter = LATITUDE_FORMATTER
            # draw cross section X
            plt.sca(ax_x)
            ax_x.contourf(max_x, trgz, cmap=cmap, levels=levels)
            ax_x.set_ylim(trgz[0], trgz[-1])
            # draw cross section Y
            plt.sca(ax_y)
            ax_y.contourf(max_y, trgz, cmap=cmap, levels=levels)
            ax_y.set_ylim(trgz[0], trgz[-1])
            # draw colorbar
            plt.sca(ax_cb)
            cb = plt.colorbar(xy, cax=ax_cb, orientation="vertical")
            cb.set_label(f'[{munit}]', fontsize=15)
            cb.ax.tick_params(labelsize=15)
            # draw corner colorbar
            plt.sca(ax_cnr)
            plt.hist2d(max_c.flatten(), max_x.flatten(), bins=100,
                       cmap=cmap, norm=LogNorm())
            plt.xlabel(f'Max-X [{munit}]', fontsize=15)
            plt.ylabel(f'Max-Y [{munit}]', fontsize=15)
            plt.title(f'{stitle}', fontsize=15)
            plt.xlabel(f'Max-X [{munit}]', fontsize=15)
            plt.ylabel(f'Max-Y [{munit}]', fontsize=15)
            plt.title(f'{stitle}', fontsize=15)
            plt.colorbar()
            plt.show()

In [20]:
import fnmatch

In [23]:
def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

In [36]:
import os
input_dir  = "../../IMD_Radar_stuff/PyScanCf/Examples/goa16/"
# Use os.scandir() to get a list of file names in the input_dir directory
files_list = [entry.name for entry in os.scandir(input_dir) if 'nc' in entry.name]

# Sort the list of file names using the natural_sort_key function
files = sorted(files_list, key=natural_sort_key)